In [2]:
lu = 250
f = open('./250/arabidopsis.csv')
ara_data = []
ara_label = []
rice_data = []
rice_label = []
non_data=[]
non_label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    if(int(tt[1]) == 1):
        ara_data.append(tt[0])
        ara_label.append(int(tt[1]))
    else:
        non_data.append(tt[0])
        non_label.append(int(tt[1]))

lu = 250
f = open('./250/rice.csv')

while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
   
    if(int(tt[1]) == 1):
        rice_data.append(tt[0])
        rice_label.append(int(tt[1])+1)
    else:
        non_data.append(tt[0])
        non_label.append(int(tt[1]))



print(len(rice_data))
print(len(ara_data))
print(len(non_data))

zong_data = rice_data + ara_data[:9156] + non_data[:9156]
zong_label = rice_label +  ara_label[:9156] + non_label[:9156]




import random



# 使用zip()函数将两个数组合并成元组的列表
combined = list(zip(zong_data, zong_label))



# 使用random.shuffle()函数对合并后的列表进行打乱
random.shuffle(combined)


# 使用zip()函数将打乱后的元组的列表拆分成两个数组
zong_data, zong_label = zip(*combined)
from sklearn.model_selection import train_test_split

train_data,test_data,train_label,test_label=train_test_split(zong_data,zong_label,test_size=0.1,random_state=3753,stratify=zong_label)

f = open('./train_duofenlei.csv','w')
f.write('seq,label\n')
data = []
for i in range(len(train_data)):
    f.write(train_data[i]+','+str(train_label[i])+'\n')

f = open('./test_duofenlei.csv','w')
f.write('seq,label\n')
data = []
for i in range(len(test_data)):
    f.write(test_data[i]+','+str(test_label[i])+'\n')



9156
19989
29145


In [8]:
train_data[i]

[[0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 0, 0,

In [ ]:
# ara  + gpt2  + EFPN


import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2ForSequenceClassification, GPT2Tokenizer
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, AdamW
import numpy as np
import evaluate
from transformers import TrainingArguments
from transformers import TrainingArguments, Trainer
import torch.nn.functional as F


import torch
import numpy as np

# 设置随机种子

torch.manual_seed(48)
np.random.seed(56)

# 加载预训练的GPT-2模型和tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("./GPT-2")



config = GPT2Config.from_pretrained("./GPT-2")

# 调整模型的层数
config.n_layer = 3  # 设置为6层
config.num_labels = 3
model = GPT2ForSequenceClassification.from_pretrained("./GPT-2",config = config)


# 获取当前wte层的权重矩阵和embedding维度
current_wte = model.transformer.wte.weight
embedding_dim = current_wte.size(1)

# 自定义一个新的嵌入层（可以替换为您想要的嵌入方式）
class CustomWte(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CustomWte, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1d1 = nn.Conv1d(embedding_dim, 384, kernel_size=3, padding=1)
        self.conv1d2 = nn.Conv1d(384, embedding_dim-1, kernel_size=3, padding=1)
        self.conv1d3 = nn.Conv1d(embedding_dim, embedding_dim, kernel_size=5, padding=2)
        self.conv1d4 = nn.Conv1d(embedding_dim, embedding_dim, kernel_size=17, padding=8)

    def forward(self, input_ids):
        embeddings = self.embedding(input_ids)
        embeddings = embeddings.transpose(1, 2)  # 将维度调整为(batch_size, emb_size, seq_len)
        conv_output1 = self.conv1d1(embeddings)

        # conv_output1 = conv_output1.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)

        conv_output1 = self.conv1d2(conv_output1)
        conv_output1 = conv_output1.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)

        conv_output2 = self.conv1d3(embeddings)
        
        conv_output2 = conv_output2.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)
        conv_output2 = F.max_pool1d(conv_output2, kernel_size=conv_output2.size(2))

        conv_output3 = self.conv1d4(embeddings)
        conv_output3 = conv_output3.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)
        conv_output3 = F.max_pool1d(conv_output3, kernel_size=conv_output3.size(2))
        
        conv_output4 = conv_output2 + conv_output3
        conv_output5 = torch.cat([conv_output1,conv_output4],dim = 2)
  
        return conv_output5

# 替换模型的wte层为自定义的嵌入层
custom_wte = CustomWte(tokenizer.vocab_size, embedding_dim)
model.transformer.wte = custom_wte

print(model)











metric = evaluate.load("./tools/accuracy.py")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


print("********")





data_file = "./data/train.csv" # 数据文件路径，数据需要提前下载
model_name = "./RBT3/RBT3"

# 加载数据集
dataset = load_dataset("csv", data_files=data_file)
dataset = dataset.filter(lambda x: x["seq"] is not None)
datasets = dataset["train"].train_test_split(0.1)

# 数据集处理
tokenizer = AutoTokenizer.from_pretrained(model_name)

def process_function(examples):
    for i in range(len(examples['seq'])):
        examples['seq'][i] = ' '.join(list(examples['seq'][i]))
    tokenized_examples = tokenizer(examples["seq"], max_length=500, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


args = TrainingArguments(
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    save_steps = 4,
    seed=4112,
    output_dir="/hy-tmp/gpt2_EFPN_HBLOSS",
    logging_steps=10,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True
)

trainer = Trainer(
  model,
  args,
  train_dataset=tokenized_datasets["train"],
  eval_dataset=tokenized_datasets["test"],
  tokenizer=tokenizer,
  compute_metrics=compute_metrics,
 data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)
trainer.train()



data_file = "./test_duofenlei.csv" # 数据文件路径，数据需要提前下载

# 加载数据集
dataset = load_dataset("csv", data_files=data_file)
dataset = dataset.filter(lambda x: x["seq"] is not None)
datasets = dataset["train"]



def process_function(examples):
    for i in range(len(examples['seq'])):
        examples['seq'][i] = ' '.join(list(examples['seq'][i]))
    tokenized_examples = tokenizer(examples["seq"], max_length=500, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

test_datasets = datasets.map(process_function, batched=True)
predictions = trainer.predict(test_datasets)
print(predictions)
trainer.save_model("./duofenleimodel/efpngpt2")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ./GPT-2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): CustomWte(
      (embedding): Embedding(50257, 768)
      (conv1d1): Conv1d(768, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv1d2): Conv1d(384, 767, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv1d3): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=(2,))
      (conv1d4): Conv1d(768, 768, kernel_size=(17,), stride=(1,), padding=(8,))
    )
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
      

Map: 100%|██████████| 425/425 [00:00<00:00, 2127.22 examples/s]
/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss


In [4]:
# ara  + gpt2  + EFPN


import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2ForSequenceClassification, GPT2Tokenizer
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, AdamW
import numpy as np
import evaluate
from transformers import TrainingArguments
from transformers import TrainingArguments, Trainer
import torch.nn.functional as F


import torch
import numpy as np

# 设置随机种子

torch.manual_seed(48)
np.random.seed(56)

# 加载预训练的GPT-2模型和tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2zzz")



config = GPT2Config.from_pretrained("./gpt2zzz")

# 调整模型的层数
config.n_layer = 5  # 设置为6层
config.num_labels = 3
model = GPT2ForSequenceClassification.from_pretrained("./gpt2zzz",config = config)


# 获取当前wte层的权重矩阵和embedding维度
current_wte = model.transformer.wte.weight
embedding_dim = current_wte.size(1)

# 自定义一个新的嵌入层（可以替换为您想要的嵌入方式）
class CustomWte(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CustomWte, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1d1 = nn.Conv1d(embedding_dim, 384, kernel_size=3, padding=1)
        self.conv1d2 = nn.Conv1d(384, embedding_dim-1, kernel_size=3, padding=1)
        self.conv1d3 = nn.Conv1d(embedding_dim, embedding_dim, kernel_size=5, padding=2)
        self.conv1d4 = nn.Conv1d(embedding_dim, embedding_dim, kernel_size=17, padding=8)

    def forward(self, input_ids):
        embeddings = self.embedding(input_ids)
        embeddings = embeddings.transpose(1, 2)  # 将维度调整为(batch_size, emb_size, seq_len)
        conv_output1 = self.conv1d1(embeddings)

        # conv_output1 = conv_output1.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)

        conv_output1 = self.conv1d2(conv_output1)
        conv_output1 = conv_output1.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)

        conv_output2 = self.conv1d3(embeddings)
        
        conv_output2 = conv_output2.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)
        conv_output2 = F.max_pool1d(conv_output2, kernel_size=conv_output2.size(2))

        conv_output3 = self.conv1d4(embeddings)
        conv_output3 = conv_output3.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)
        conv_output3 = F.max_pool1d(conv_output3, kernel_size=conv_output3.size(2))
        
        conv_output4 = conv_output2 + conv_output3
        conv_output5 = torch.cat([conv_output1,conv_output4],dim = 2)
  
        return conv_output5

# 替换模型的wte层为自定义的嵌入层
custom_wte = CustomWte(tokenizer.vocab_size, embedding_dim)
model.transformer.wte = custom_wte

print(model)











metric = evaluate.load("./evaluate/metrics/accuracy/accuracy.py")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


print("********")





data_file = "./train_duofenlei.csv" # 数据文件路径，数据需要提前下载
model_name = "./rbt3"

# 加载数据集
dataset = load_dataset("csv", data_files=data_file)
dataset = dataset.filter(lambda x: x["seq"] is not None)
datasets = dataset["train"].train_test_split(0.1)

# 数据集处理
tokenizer = AutoTokenizer.from_pretrained(model_name)

def process_function(examples):
    for i in range(len(examples['seq'])):
        examples['seq'][i] = ' '.join(list(examples['seq'][i]))
    tokenized_examples = tokenizer(examples["seq"], max_length=500, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


args = TrainingArguments(
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    save_steps = 4,
    seed=4112,
    output_dir="/hy-tmp/gpt2_EFPN_HBLOSS",
    logging_steps=10,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True
)

trainer = Trainer(
  model,
  args,
  train_dataset=tokenized_datasets["train"],
  eval_dataset=tokenized_datasets["test"],
  tokenizer=tokenizer,
  compute_metrics=compute_metrics,
 data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)
trainer.train()



data_file = "./test_duofenlei.csv" # 数据文件路径，数据需要提前下载

# 加载数据集
dataset = load_dataset("csv", data_files=data_file)
dataset = dataset.filter(lambda x: x["seq"] is not None)
datasets = dataset["train"]



def process_function(examples):
    for i in range(len(examples['seq'])):
        examples['seq'][i] = ' '.join(list(examples['seq'][i]))
    tokenized_examples = tokenizer(examples["seq"], max_length=500, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

test_datasets = datasets.map(process_function, batched=True)
predictions = trainer.predict(test_datasets)
print(predictions)
trainer.save_model("./duofenleimodel/efpngpt2")

Didn't find file ./gpt2zzz/added_tokens.json. We won't load it.
Didn't find file ./gpt2zzz/special_tokens_map.json. We won't load it.
loading file ./gpt2zzz/vocab.json
loading file ./gpt2zzz/merges.txt
loading file None
loading file None
loading file ./gpt2zzz/tokenizer_config.json
loading configuration file ./gpt2zzz/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt2zzz",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 3,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_ty

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): CustomWte(
      (embedding): Embedding(50257, 768)
      (conv1d1): Conv1d(768, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv1d2): Conv1d(384, 767, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv1d3): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=(2,))
      (conv1d4): Conv1d(768, 768, kernel_size=(17,), stride=(1,), padding=(8,))
    )
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
      

loading configuration file ./rbt3/config.json
Model config BertConfig {
  "_name_or_path": "./rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

loading file ./rbt3/vocab.txt
loading file ./rbt3/tokenizer.json
loading file ./rbt3/added_tokens.json
loading file ./rbt3/spe

Epoch,Training Loss,Validation Loss,Accuracy
1,0.803400,0.747240,0.661949
2,0.774800,0.723604,0.687829
3,0.541400,0.630797,0.736757
4,0.657900,0.628735,0.742014
5,0.568300,0.600187,0.763041
6,0.549400,0.601766,0.774767
7,0.526200,0.548829,0.780024
8,0.424000,0.539408,0.785281
9,0.506700,0.552410,0.791751
10,0.463300,0.591488,0.771128


The following columns in the evaluation set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2473
  Batch size = 16
Saving model checkpoint to /hy-tmp/gpt2_EFPN_HBLOSS/checkpoint-1391
Configuration saved in /hy-tmp/gpt2_EFPN_HBLOSS/checkpoint-1391/config.json
Model weights saved in /hy-tmp/gpt2_EFPN_HBLOSS/checkpoint-1391/pytorch_model.bin
tokenizer config file saved in /hy-tmp/gpt2_EFPN_HBLOSS/checkpoint-1391/tokenizer_config.json
Special tokens file saved in /hy-tmp/gpt2_EFPN_HBLOSS/checkpoint-1391/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***

Saving model checkpoint to ./duofenleimodel/efpngpt2
Configuration saved in ./duofenleimodel/efpngpt2/config.json


PredictionOutput(predictions=array([[-3.238 ,  3.56  ,  1.027 ],
       [-2.576 ,  3.098 ,  1.573 ],
       [-3.355 , -2.791 ,  6.55  ],
       ...,
       [-1.208 , -1.341 ,  2.27  ],
       [ 2.494 ,  0.3867, -2.12  ],
       [ 4.03  , -1.148 , -2.578 ]], dtype=float16), label_ids=array([1, 1, 2, ..., 2, 1, 0]), metrics={'test_loss': 0.4935366213321686, 'test_accuracy': 0.7986894794321078, 'test_runtime': 40.1317, 'test_samples_per_second': 68.45, 'test_steps_per_second': 4.286})


Model weights saved in ./duofenleimodel/efpngpt2/pytorch_model.bin
tokenizer config file saved in ./duofenleimodel/efpngpt2/tokenizer_config.json
Special tokens file saved in ./duofenleimodel/efpngpt2/special_tokens_map.json


In [8]:
print(len(tokenized_datasets["train"]))

22248


In [10]:
trainer.save_model("./duofenleimodel/efpngpt2")

Saving model checkpoint to ./duofenleimodel/efpngpt2
Configuration saved in ./duofenleimodel/efpngpt2/config.json
Model weights saved in ./duofenleimodel/efpngpt2/pytorch_model.bin
tokenizer config file saved in ./duofenleimodel/efpngpt2/tokenizer_config.json
Special tokens file saved in ./duofenleimodel/efpngpt2/special_tokens_map.json


In [ ]:
# rbt3 duofenlei

#rbt3   ara EFPN

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2ForSequenceClassification, GPT2Tokenizer
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, AdamW
import numpy as np
import evaluate
from transformers import TrainingArguments
from transformers import TrainingArguments, Trainer
import torch.nn.functional as F


import torch
import numpy as np


data_file = "./train_duofenlei.csv" # 数据文件路径，数据需要提前下载
model_name = "./rbt3" # 所使用模型

# 加载数据集
dataset = load_dataset("csv", data_files=data_file)
dataset = dataset.filter(lambda x: x["seq"] is not None)
datasets = dataset["train"].train_test_split(0.1)

# 数据集处理
tokenizer = AutoTokenizer.from_pretrained(model_name)

def process_function(examples):
    for i in range(len(examples['seq'])):
        examples['seq'][i] = ' '.join(list(examples['seq'][i]))
    tokenized_examples = tokenizer(examples["seq"], max_length=500, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True)










# 构建评估函数
metric = evaluate.load('./evaluate/metrics/accuracy/accuracy.py')

def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 训练器配置
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)





print(model)





# 获取当前wte层的权重矩阵和embedding维度
current_wte = model.bert.embeddings.word_embeddings.weight
embedding_dim = current_wte.size(1)

# 自定义一个新的嵌入层（可以替换为您想要的嵌入方式）
class CustomWte(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CustomWte, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1d1 = nn.Conv1d(embedding_dim, 384, kernel_size=3, padding=1)
        self.conv1d2 = nn.Conv1d(384, embedding_dim-1, kernel_size=3, padding=1)
        self.conv1d3 = nn.Conv1d(embedding_dim, embedding_dim, kernel_size=5, padding=2)
        self.conv1d4 = nn.Conv1d(embedding_dim, embedding_dim, kernel_size=17, padding=8)

    def forward(self, input_ids):
        embeddings = self.embedding(input_ids)
        embeddings = embeddings.transpose(1, 2)  # 将维度调整为(batch_size, emb_size, seq_len)
        conv_output1 = self.conv1d1(embeddings)

        # conv_output1 = conv_output1.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)

        conv_output1 = self.conv1d2(conv_output1)
        conv_output1 = conv_output1.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)

        conv_output2 = self.conv1d3(embeddings)
        
        conv_output2 = conv_output2.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)
        conv_output2 = F.max_pool1d(conv_output2, kernel_size=conv_output2.size(2))

        conv_output3 = self.conv1d4(embeddings)
        conv_output3 = conv_output3.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)
        conv_output3 = F.max_pool1d(conv_output3, kernel_size=conv_output3.size(2))
        
        conv_output4 = conv_output2 + conv_output3
        conv_output5 = torch.cat([conv_output1,conv_output4],dim = 2)
  
        return conv_output5

# 替换模型的wte层为自定义的嵌入层
custom_wte = CustomWte(tokenizer.vocab_size, embedding_dim)
model.bert.embeddings.word_embeddings = custom_wte

print(model)


























args = TrainingArguments(
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=20,
  weight_decay=0.01,
        save_steps = 4,
    seed=4112,
  output_dir="/hy-tmp/bert3_ara_classification",
  logging_steps=10,
  evaluation_strategy = "epoch",
  save_strategy = "epoch",
  load_best_model_at_end=True,
  metric_for_best_model="accuracy",
  fp16=True
)

trainer = Trainer(
  model,
  args,
  train_dataset=tokenized_datasets["train"],
  eval_dataset=tokenized_datasets["test"],
  tokenizer=tokenizer,
  compute_metrics=compute_metrics,
  data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

# 训练与评估
trainer.train()
data_file = "./test_duofenlei.csv" # 数据文件路径，数据需要提前下载

# 加载数据集
dataset = load_dataset("csv", data_files=data_file)
dataset = dataset.filter(lambda x: x["seq"] is not None)
datasets = dataset["train"]



def process_function(examples):
    for i in range(len(examples['seq'])):
        examples['seq'][i] = ' '.join(list(examples['seq'][i]))
    tokenized_examples = tokenizer(examples["seq"], max_length=500, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

test_datasets = datasets.map(process_function, batched=True)

trainer.save_model("./duofenleimodel/bert3ara_EFPN")
# trainer.evaluate()
predictions = trainer.predict(test_datasets)
print(predictions)

loading configuration file ./rbt3/config.json
Model config BertConfig {
  "_name_or_path": "./rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

loading file ./rbt3/vocab.txt
loading file ./rbt3/tokenizer.json
loading file ./rbt3/added_tokens.json
loading file ./rbt3/spe

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 22248
  Num Epochs = 20
  Instantaneous batch s

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): CustomWte(
        (embedding): Embedding(21128, 768)
        (conv1d1): Conv1d(768, 384, kernel_size=(3,), stride=(1,), padding=(1,))
        (conv1d2): Conv1d(384, 767, kernel_size=(3,), stride=(1,), padding=(1,))
        (conv1d3): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=(2,))
        (conv1d4): Conv1d(768, 768, kernel_size=(17,), stride=(1,), padding=(8,))
      )
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, o

Epoch,Training Loss,Validation Loss,Accuracy
1,0.623200,0.710100,0.724626
2,0.605700,0.579693,0.763850
3,0.544500,0.671057,0.749292
4,0.581100,0.630538,0.756167
5,0.469700,0.595819,0.771533
6,0.527900,0.508993,0.800647
7,0.526100,0.600525,0.771533
8,0.406200,0.545181,0.788516


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2473
  Batch size = 16
Saving model checkpoint to /hy-tmp/bert3_ara_classification/checkpoint-1391
Configuration saved in /hy-tmp/bert3_ara_classification/checkpoint-1391/config.json
Model weights saved in /hy-tmp/bert3_ara_classification/checkpoint-1391/pytorch_model.bin
tokenizer config file saved in /hy-tmp/bert3_ara_classification/checkpoint-1391/tokenizer_config.json
Special tokens file saved in /hy-tmp/bert3_ara_classification/checkpoint-1391/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `BertForSequenceClassification.forward`, 

In [3]:
# rbt3 duofenlei

#rbt3   ara EFPN

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2ForSequenceClassification, GPT2Tokenizer
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, AdamW
import numpy as np
import evaluate
from transformers import TrainingArguments
from transformers import TrainingArguments, Trainer
import torch.nn.functional as F


import torch
import numpy as np


data_file = "./train_duofenlei.csv" # 数据文件路径，数据需要提前下载
model_name = "./rbt3" # 所使用模型

# 加载数据集
dataset = load_dataset("csv", data_files=data_file)
dataset = dataset.filter(lambda x: x["seq"] is not None)
datasets = dataset["train"].train_test_split(0.1)

# 数据集处理
tokenizer = AutoTokenizer.from_pretrained(model_name)

def process_function(examples):
    for i in range(len(examples['seq'])):
        examples['seq'][i] = ' '.join(list(examples['seq'][i]))
    tokenized_examples = tokenizer(examples["seq"], max_length=500, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True)










# 构建评估函数
metric = evaluate.load('./evaluate/metrics/accuracy/accuracy.py')

def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 训练器配置
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)





print(model)





# 获取当前wte层的权重矩阵和embedding维度
current_wte = model.bert.embeddings.word_embeddings.weight
embedding_dim = current_wte.size(1)

# 自定义一个新的嵌入层（可以替换为您想要的嵌入方式）
class CustomWte(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CustomWte, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1d1 = nn.Conv1d(embedding_dim, 384, kernel_size=3, padding=1)
        self.conv1d2 = nn.Conv1d(384, embedding_dim-1, kernel_size=3, padding=1)
        self.conv1d3 = nn.Conv1d(embedding_dim, embedding_dim, kernel_size=5, padding=2)
        self.conv1d4 = nn.Conv1d(embedding_dim, embedding_dim, kernel_size=17, padding=8)

    def forward(self, input_ids):
        embeddings = self.embedding(input_ids)
        embeddings = embeddings.transpose(1, 2)  # 将维度调整为(batch_size, emb_size, seq_len)
        conv_output1 = self.conv1d1(embeddings)

        # conv_output1 = conv_output1.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)

        conv_output1 = self.conv1d2(conv_output1)
        conv_output1 = conv_output1.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)

        conv_output2 = self.conv1d3(embeddings)
        
        conv_output2 = conv_output2.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)
        conv_output2 = F.max_pool1d(conv_output2, kernel_size=conv_output2.size(2))

        conv_output3 = self.conv1d4(embeddings)
        conv_output3 = conv_output3.transpose(1, 2)  # 将维度调整为(batch_size, seq_len, emb_size)
        conv_output3 = F.max_pool1d(conv_output3, kernel_size=conv_output3.size(2))
        
        conv_output4 = conv_output2 + conv_output3
        conv_output5 = torch.cat([conv_output1,conv_output4],dim = 2)
  
        return conv_output5

# 替换模型的wte层为自定义的嵌入层
custom_wte = CustomWte(tokenizer.vocab_size, embedding_dim)
model.bert.embeddings.word_embeddings = custom_wte

print(model)


























args = TrainingArguments(
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=20,
  weight_decay=0.01,
        save_steps = 4,
    seed=4112,
  output_dir="/hy-tmp/bert3_ara_classification",
  logging_steps=10,
  evaluation_strategy = "epoch",
  save_strategy = "epoch",
  load_best_model_at_end=True,
  metric_for_best_model="accuracy",
  fp16=True
)

trainer = Trainer(
  model,
  args,
  train_dataset=tokenized_datasets["train"],
  eval_dataset=tokenized_datasets["test"],
  tokenizer=tokenizer,
  compute_metrics=compute_metrics,
  data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

# 训练与评估
trainer.train()
data_file = "./test_duofenlei.csv" # 数据文件路径，数据需要提前下载

# 加载数据集
dataset = load_dataset("csv", data_files=data_file)
dataset = dataset.filter(lambda x: x["seq"] is not None)
datasets = dataset["train"]



def process_function(examples):
    for i in range(len(examples['seq'])):
        examples['seq'][i] = ' '.join(list(examples['seq'][i]))
    tokenized_examples = tokenizer(examples["seq"], max_length=500, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

test_datasets = datasets.map(process_function, batched=True)

trainer.save_model("./duofenleimodel/bert3ara_EFPN")
# trainer.evaluate()
predictions = trainer.predict(test_datasets)
print(predictions)

loading configuration file ./rbt3/config.json
Model config BertConfig {
  "_name_or_path": "./rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

loading file ./rbt3/vocab.txt
loading file ./rbt3/tokenizer.json
loading file ./rbt3/added_tokens.json
loading file ./rbt3/spe

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 22248
  Num Epochs = 20
  Instantaneous batch s

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): CustomWte(
        (embedding): Embedding(21128, 768)
        (conv1d1): Conv1d(768, 384, kernel_size=(3,), stride=(1,), padding=(1,))
        (conv1d2): Conv1d(384, 767, kernel_size=(3,), stride=(1,), padding=(1,))
        (conv1d3): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=(2,))
        (conv1d4): Conv1d(768, 768, kernel_size=(17,), stride=(1,), padding=(8,))
      )
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, o

Epoch,Training Loss,Validation Loss,Accuracy
1,0.713200,0.694936,0.720582
2,0.642500,0.749093,0.690659
3,0.580200,0.558500,0.775576
4,0.581100,0.580652,0.763850
5,0.472500,0.617078,0.752123
6,0.489900,0.599748,0.781237
7,0.511900,0.670981,0.736757
8,0.515700,0.506806,0.809543
9,0.472200,0.678833,0.763041
10,0.464000,0.510538,0.803882


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2473
  Batch size = 16
Saving model checkpoint to /hy-tmp/bert3_ara_classification/checkpoint-1391
Configuration saved in /hy-tmp/bert3_ara_classification/checkpoint-1391/config.json
Model weights saved in /hy-tmp/bert3_ara_classification/checkpoint-1391/pytorch_model.bin
tokenizer config file saved in /hy-tmp/bert3_ara_classification/checkpoint-1391/tokenizer_config.json
Special tokens file saved in /hy-tmp/bert3_ara_classification/checkpoint-1391/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `BertForSequenceClassification.forward`, 

PredictionOutput(predictions=array([[-3.25   ,  2.662  ,  0.8506 ],
       [-3.912  ,  3.018  ,  0.8545 ],
       [-2.834  , -1.954  ,  4.297  ],
       ...,
       [-1.667  , -2.219  ,  3.523  ],
       [ 0.06824,  0.394  , -0.6245 ],
       [ 3.617  , -2.621  , -1.481  ]], dtype=float16), label_ids=array([1, 1, 2, ..., 2, 1, 0]), metrics={'test_loss': 0.5371641516685486, 'test_accuracy': 0.8150709865307608, 'test_runtime': 21.3388, 'test_samples_per_second': 128.733, 'test_steps_per_second': 8.06})
